In [ ]:
!pip install pyspellchecker transformers groq tiktoken verovio ipywidgets pillow python-docx fpdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 74.9 MB/s eta 0:00:00


In [ ]:
!pip install indexer==0.6.2

  Using cached indexer-0.6.2.tar.gz (14 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
from spellchecker import SpellChecker
from transformers import AutoModel, AutoTokenizer
import re

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('ucaslcl/GOT-OCR2_0', trust_remote_code=True)
model = AutoModel.from_pretrained('ucaslcl/GOT-OCR2_0', trust_remote_code=True, low_cpu_mem_usage=True, device_map='cuda', use_safetensors=True, pad_token_id=tokenizer.eos_token_id)
model = model.eval().cuda()

qwen.tiktoken:   0%|          | 0.00/2.56M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/149 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/986 [00:00<?, ?B/s]

modeling_GOT.py:   0%|          | 0.00/33.8k [00:00<?, ?B/s]

got_vision_b.py:   0%|          | 0.00/16.1k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ucaslcl/GOT-OCR2_0:
- got_vision_b.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


render_tools.py:   0%|          | 0.00/1.99k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ucaslcl/GOT-OCR2_0:
- render_tools.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/ucaslcl/GOT-OCR2_0:
- modeling_GOT.py
- got_vision_b.py
- render_tools.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors:   0%|          | 0.00/1.43G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

In [ ]:
def correct_spelling(text):
    """
    Corrects spelling errors in the text while preserving the original structure.
    """
    spell = SpellChecker()
    corrected_text = []

    # Split the text into lines to preserve line breaks
    lines = text.split("\n")

    for line in lines:
        corrected_line = []
        words = line.split()  # Split each line into words

        for word in words:
            # Correct the word if it's misspelled
            corrected_word = spell.correction(word)
            # Preserve the original word if the corrected word is None (e.g., for punctuation)
            corrected_line.append(corrected_word if corrected_word is not None else word)

        # Join the corrected words back into a line
        corrected_text.append(" ".join(corrected_line))

    # Join the lines back into a single text
    return "\n".join(corrected_text)

In [ ]:
def add_space_after_punctuation(text):
    # Regex to match punctuation marks and add a space after them
    corrected_text = re.sub(r'([.,!?;:])(?![ \n])', r'\1 ', text)
    return corrected_text

In [ ]:
from groq import Groq
api_key = "YOUR_GROQ_API"

# Set the API key here
client = Groq(api_key=api_key)

def clean_text(corrected_text):
    """
    Uses Groq's API to clean and format text by removing unnecessary content.

    Parameters:
        input_text (str): The raw text input to be cleaned.

    Returns:
        str: The cleaned and formatted text.
    """
    try:
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "system",
                    "content": (
                        "You are a helpful assistant. Text will be provided to you, "
                        "you need to remove the unnecessary content out of it which "
                        "doesn't make sense and keep the actual text out of it. "
                        "Just give the formatted text with perfect format. Nothing else."
                        "Follow the paragraph architecture and indetations if necessary."
                    ),
                },
                {
                    "role": "user",
                    "content": corrected_text,
                }
            ],
            model="llama-3.3-70b-versatile",
            temperature=0.5,
            max_completion_tokens=1024,
            top_p=1,
            stop=None,
            stream=False,
        )

        return chat_completion.choices[0].message.content

    except Exception as e:
        return f"Error: {str(e)}"

#


In [ ]:
from transformers import logging
logging.set_verbosity_error()


In [ ]:
import ipywidgets as widgets
from IPython.display import display, HTML
import io
from PIL import Image
import base64
from docx import Document
from fpdf import FPDF

In [46]:
import ipywidgets as widgets
from IPython.display import display, HTML
import base64
import io
from PIL import Image
from docx import Document
from fpdf import FPDF

def create_ocr_gui():
    # File Upload
    file_upload = widgets.FileUpload(
        accept='image/*',
        multiple=False,
        description='📂 Upload Image',
        button_style='primary'
    )

    # Image Display
    image_output = widgets.Output(layout={'border': '1px solid #ccc', 'padding': '10px', 'width': '45%'})

    # Extracted Text Display
    text_output = widgets.Output(layout={'border': '1px solid #ccc', 'padding': '20px', 'width': '50%', 'height': '300px', 'overflow': 'auto'})

    # Dropdown for download format
    download_format = widgets.Dropdown(
        options=['txt', 'docx', 'pdf'],
        value='txt',
        description='📑 Format:',
        disabled=False
    )

    # Download Button
    download_button = widgets.Button(
        description='⬇️ Download',
        disabled=True,
        button_style='success'
    )

    def save_as_docx(text, filename):
        doc = Document()
        doc.add_paragraph(text)
        doc.save(filename)

    def save_as_pdf(text, filename):
        pdf = FPDF()
        pdf.add_page()
        pdf.set_font("Arial", size=12)
        pdf.multi_cell(0, 10, txt=text)
        pdf.output(filename)

    def on_file_upload(change):
        if change['type'] == 'change' and change['name'] == 'value':
            image_output.clear_output()
            text_output.clear_output()

            uploaded_file = list(change['new'].values())[0]
            content = uploaded_file['content']

            with image_output:
                image = Image.open(io.BytesIO(content))
                img_byte_arr = io.BytesIO()
                image.save(img_byte_arr, format=image.format)
                img_base64 = base64.b64encode(img_byte_arr.getvalue()).decode()
                display(HTML(f'<img src="data:image/jpeg;base64,{img_base64}" width="100%"/>'))

            # Process the image with your OCR model
            with open("/tmp/temp_image.png", "wb") as f:
                f.write(content)

            # Use OCR function (Replace this with your actual OCR processing)
            res = model.chat(tokenizer, "/tmp/temp_image.png", ocr_type='ocr')
            corrected_text = correct_spelling(res)
            final_text = add_space_after_punctuation(corrected_text)
            extracted_text = clean_text(final_text)

            with text_output:
                print(extracted_text)

            download_button.result = extracted_text
            download_button.disabled = False  # Enable Download Button

    def on_download_clicked(b):
        if hasattr(download_button, 'result'):
            format_type = download_format.value
            filename = f'extracted_text.{format_type}'

            if format_type == 'txt':
                with open(filename, 'w', encoding='utf-8') as f:
                    f.write(download_button.result)
            elif format_type == 'docx':
                save_as_docx(download_button.result, filename)
            elif format_type == 'pdf':
                save_as_pdf(download_button.result, filename)

            with open(filename, 'rb') as f:
                content = f.read()

            display(HTML(
                f'<a href="data:application/{format_type};base64,{base64.b64encode(content).decode()}" '
                f'download="{filename}" style="font-size:16px; color:blue;">📥 Click to download {filename}</a>'
            ))

    # Register callbacks
    file_upload.observe(on_file_upload)
    download_button.on_click(on_download_clicked)

    # Layout
    ui_layout = widgets.VBox([
        widgets.HTML('<h3 style="text-align:center; color:#FFF; text-shadow: 1px 1px 2px #000;">📄 OCR Image to Text Extractor</h3>'),
        widgets.HBox([file_upload], layout=widgets.Layout(justify_content='center', margin='10px')),
        widgets.HBox([image_output, text_output], layout=widgets.Layout(justify_content='space-between')),
        widgets.HBox([download_format, download_button], layout=widgets.Layout(justify_content='center', margin='20px'))
    ])

    display(ui_layout)

# Run the GUI
create_ocr_gui()
